In [ ]:
# input image 224*224*3
# feature vector 1*1*2048

In [ ]:
#we'll use inception model for now!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class EncoderCNN(nn.Module):
  def __init__(self, embed_size, train_CNN=False):
    super(EncoderCNN, self).__init__()
    self.train_CNN = train_CNN
    #now here we gone import pretrend cnn and use tranfer learning
    self.inception = models.inception_v3(pretrained=True, aux_logits=False)
    self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
    self.relu = nn.ReLU()
    self.dropout = nn.dropout(0.5)

  def forward(self, images):
    features = self.inception(images)

    #fine tuning
    for name, param in self.inception.named_parameters():
      if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
      else:
        param.requires_grad = self.train_CNN
    
    return self.dropout(self.relu(features))

#NOW for RNN
class DecoderRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(DecoderRNN, self).__init__() 
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, vocab_size) #hidden layer is output of privius lstm layer
    self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions):
    embeddings = self.dropout(self.embed(captions))
    embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0) #concatinate
    hiddens, _ = self.lstm(embeddings)
    outputs = self.linear(hiddens)
    return outputs
  


  
class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(CNNtoCNN, self).__init__()
    self.encoderCNN = EncoderCNN(embed_size)
    self.decodeRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

  def forward(self, images, captions):
    features = self.encoderCNN(images)
    outputs = self.decoderRNN(features, captions)
    return outputs

  def caption_image(self, image, vocabulary, max_length=1000):
      result_caption = []

      with torch.no_grad():
          x = self.encoderCNN(image).unsqueeze(0)
          states = None #hidden state for the lstm, in the begining states are none, but leter on it we be change.

          for _ in range(max_length):
              hiddens, states = self.decoderRNN.lstm(x, states)
              output = self.decoderRNN.linear(hiddens.squeeze(0))
              predicted =output.argmax(1) 

              result_caption.append(predicted.item())
              x = self.decoderRNN.embed(predicted).unsqueeze(0)
     
              if vocabulary.itos[predicted.item()] == "<EOS>":
                  break
      return [vocabulary.itos[idx] for idx in result_caption]     

In [ ]:
 #traing the model
#!pip install utils
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from utils import save_checkpoint, load_checkpoint, print_examples
from loader import get_loader
#from model import CNNtoRNN

def train():
  transform = transforms.Compose(
      [
          transforms.Resize((356, 356)),
          transforms.RandomCrop((299, 299)),
          transforms.ToTensor(),
          transforms.Normalizer((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
      ]
  )

  train_loader, dataset=get_loader(
      root_folder ="/content/drive/My Drive/Colab Notebooks (1)/public_test_images",
      annotation_file = "/content/drive/My Drive/Colab Notebooks (1)/caption data/txt_public_test_captions.txt",
      transform = transform,
      num_workers = 2,
  )

  tourch.backends,cudnn.benchmark = True
  device = torch.device("cuda" if torch.cuda.is_avaliable() else "cpu")
  load_model  =False
  save_model  =True

  #hyperperameter
  embed_size = 256
  hidden_size = 256
  vocab_size = len(dataset.vocab)
  num_layers = 1
  learning_rate = 3e-4
  num_epochs = 100

  #tensorboard
  writer = SummarWriter("path")
  step = 0

  #initialize model, loss and all
  model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
  criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  if load_model:
    step = load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)
  
  model.train()

  for epoch in range(num_epochs):
      print_examples(model, device, dataset)

      if save_model:
        checkpoint = {
            "state_dict" : model.state_dict(),
            "optimizer" : optimizer.state_dict(),
            "step":step, 
        }
        save_checkpoint(checkpoint)

      for idx, (imgs, caption) in enumerate(train_loader):
         imgs = imgs.to(device)
         captions = captions.to(device)  

         outputs = model(imgs, captions[:-1])
         loss = critierion(output.reshape(-1, outputs.shape[2]), caption.reshape(-1))

         writer.add_scalar("Training loss", loss.item(), global_step=step)
         step += 1

         optimizer.zero_gard()
         loss.backward(loss)
         optimizer.step()


if __name__ == "__main__":
  train()


ModuleNotFoundError: ignored

In [ ]:
# Build the neural network, expand on top of nn.Module
class Network(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)

  # define forward function
  def forward(self, t):
    # conv 1
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    # conv 2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    # fc1
    t = t.reshape(-1, 12*4*4)
    t = self.fc1(t)
    t = F.relu(t)

    # fc2
    t = self.fc2(t)
    t = F.relu(t)

    # output
    t = self.out(t)
    # don't need softmax here since we'll use cross-entropy as activation.

    return t

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

#batch_size = 500   # Number of samples in each batch
#epoch_num = 4      # Number of epochs to train the network
#lr = 0.0005        # Learning rate


class CHHIPA(nn.Module):
    def __init__(self, n_classes):
        super(CHHIPA, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(7*7*512, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 1000)

    def forward(self, x, training=True):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.pool(x)
        x = x.view(-1, 7 * 7 * 512)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)
        return x

# define the CNN and move the network into GPU
#vgg16 = VGG16(10)
#vgg16.cuda()
#from torchvision import models
model = CHHIPA(10)
print(model)


import torch
import torch.nn as nn
import torchvision.models as models

class EncoderCNN(nn.Module):
  def __init__(self, embed_size, train_CNN=False):
    super(EncoderCNN, self).__init__()
    self.train_CNN = train_CNN
    #now here we gone import pretrend cnn and use tranfer learning
    self.inception = models.inception_v3(pretrained=True, aux_logits=False)
    self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
    self.relu = nn.ReLU()
    self.dropout = nn.dropout(0.5)

  def forward(self, images):
    features = self.inception(images)

    #fine tuning
    for name, param in self.inception.named_parameters():
      if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
      else:
        param.requires_grad = self.train_CNN
    
    return self.dropout(self.relu(features))

#NOW for RNN
class DecoderRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(DecoderRNN, self).__init__() 
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, vocab_size) #hidden layer is output of privius lstm layer
    self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions):
    embeddings = self.dropout(self.embed(captions))
    embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0) #concatinate
    hiddens, _ = self.lstm(embeddings)
    outputs = self.linear(hiddens)
    return outputs
  


  
class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(CNNtoCNN, self).__init__()
    self.encoderCNN = EncoderCNN(embed_size)
    self.decodeRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

  def forward(self, images, captions):
    features = self.encoderCNN(images)
    outputs = self.decoderRNN(features, captions)
    return outputs

  def caption_image(self, image, vocabulary, max_length=1000):
      result_caption = []

      with torch.no_grad():
          x = self.encoderCNN(image).unsqueeze(0)
          states = None #hidden state for the lstm, in the begining states are none, but leter on it we be change.

          for _ in range(max_length):
              hiddens, states = self.decoderRNN.lstm(x, states)
              output = self.decoderRNN.linear(hiddens.squeeze(0))
              predicted =output.argmax(1) 

              result_caption.append(predicted.item())
              x = self.decoderRNN.embed(predicted).unsqueeze(0)
     
              if vocabulary.itos[predicted.item()] == "<EOS>":
                  break
      return [vocabulary.itos[idx] for idx in result_caption]     

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
    #now here we gone import pretrend cnn and use tranfer learning ye'ah and i'm gonna chage it. FUCK YOU
    # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
 
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(7*7*512, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 1000)


    def forward(self, x, training=True):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.pool(x)
        x = x.view(-1, 7 * 7 * 512)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)
        return x

#NOW for RNN
class DecoderRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(DecoderRNN, self).__init__() 
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, vocab_size) #hidden layer is output of privius lstm layer
    self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions):
    embeddings = self.dropout(self.embed(captions))
    embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0) #concatinate
    hiddens, _ = self.lstm(embeddings)
    outputs = self.linear(hiddens)
    return outputs
  


  
class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(CNNtoCNN, self).__init__()
    self.encoderCNN = EncoderCNN(embed_size)
    self.decodeRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

  def forward(self, images, captions):
    features = self.encoderCNN(images)
    outputs = self.decoderRNN(features, captions)
    return outputs

  def caption_image(self, image, vocabulary, max_length=1000):
      result_caption = []

      with torch.no_grad():
          x = self.encoderCNN(image).unsqueeze(0)
          states = None #hidden state for the lstm, in the begining states are none, but leter on it we be change.

          for _ in range(max_length):
              hiddens, states = self.decoderRNN.lstm(x, states)
              output = self.decoderRNN.linear(hiddens.squeeze(0))
              predicted =output.argmax(1) 

              result_caption.append(predicted.item())
              x = self.decoderRNN.embed(predicted).unsqueeze(0)
     
              if vocabulary.itos[predicted.item()] == "<EOS>":
                  break
      return [vocabulary.itos[idx] for idx in result_caption]     

In [ ]:











#hmmmmmm

import torch
import torch.nn as nn
import torchvision.models as models

class EncoderCNN(nn.Module):
  def __init__(self, embed_size, train_CNN=False):
    super(EncoderCNN, self).__init__()
    self.train_CNN = train_CNN
    #now here we gone import pretrend cnn and use tranfer learning
    self.inception = models.inception_v3(pretrained=True, aux_logits=False)
    self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
    self.relu = nn.ReLU()
    self.dropout = nn.dropout(0.5)

  def forward(self, images):
    features = self.inception(images)

    #fine tuning
    for name, param in self.inception.named_parameters():
      if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
      else:
        param.requires_grad = self.train_CNN
    
    return self.dropout(self.relu(features))

#NOW for RNN
class DecoderRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(DecoderRNN, self).__init__() 
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, vocab_size) #hidden layer is output of privius lstm layer
    self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions):
    embeddings = self.dropout(self.embed(captions))
    embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0) #concatinate
    hiddens, _ = self.lstm(embeddings)
    outputs = self.linear(hiddens)
    return outputs
  


  
class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(CNNtoCNN, self).__init__()
    self.encoderCNN = EncoderCNN(embed_size)
    self.decodeRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

  def forward(self, images, captions):
    features = self.encoderCNN(images)
    outputs = self.decoderRNN(features, captions)
    return outputs

  def caption_image(self, image, vocabulary, max_length=1000):
      result_caption = []

      with torch.no_grad():
          x = self.encoderCNN(image).unsqueeze(0)
          states = None #hidden state for the lstm, in the begining states are none, but leter on it we be change.

          for _ in range(max_length):
              hiddens, states = self.decoderRNN.lstm(x, states)
              output = self.decoderRNN.linear(hiddens.squeeze(0))
              predicted =output.argmax(1) 

              result_caption.append(predicted.item())
              x = self.decoderRNN.embed(predicted).unsqueeze(0)
     
              if vocabulary.itos[predicted.item()] == "<EOS>":
                  break
      return [vocabulary.itos[idx] for idx in result_caption]     








class ImageCaptionsNet(nn.Module):
    def __init__(self):
    #def __init__(self, input_size, hidden_size):
        super(ImageCaptionsNet, self).__init__()

        # Define your architecture here

    def forward(self, x):
        x = image_batch, captions_batch

        # Forward Propogation

        return captions_batch

net = ImageCaptionsNet()

# If GPU training is required
# net = net.cuda()













class ImageEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ImageEncoder, self).__init__()
        self.out = nn.Linear(input_size, hidden_size)
        
    def forward(self, inputs):
        return self.out(inputs)
    
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        output = F.relu(input)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = F.log_softmax(output.squeeze())
        return output.unsqueeze(0), hidden
    
encoder = ImageEncoder(input_size=4096, hidden_size=300).cuda()
decoder = DecoderLSTM(input_size=len(vocabulary), hidden_size=300, output_size=len(vocabulary)).cuda()







class ImageCaptionsNet(nn.Module):
    def __init__(self):
    #def __init__(self, input_size, hidden_size):
        super(ImageCaptionsNet, self).__init__()

        # Define your architecture here

    def forward(self, x):
        x = image_batch, captions_batch

        # Forward Propogation

        return captions_batch

net = ImageCaptionsNet()

# If GPU training is required
# net = net.cuda()


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class EncoderCNN(nn.Module):
  def __init__(self, embed_size, train_CNN=False):
    super(EncoderCNN, self).__init__()
    self.train_CNN = train_CNN
    #now here we gone import pretrend cnn and use tranfer learning
    self.inception = models.inception_v3(pretrained=True, aux_logits=False)
    self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)

  def forward(self, image_batch):
    features = self.inception(image_batch)

    #fine tuning
    for name, param in self.inception.named_parameters():
      if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
      else:
        param.requires_grad = self.train_CNN
    
    return features

#NOW for RNN
class DecoderRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
    super(DecoderRNN, self).__init__() 
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
    self.linear = nn.Linear(hidden_size, vocab_size) #hidden layer is output of privius lstm layer
    self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions_batch):
    embeddings = self.dropout(self.embed(captions_batch))
    embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0) #concatinate
    hiddens, _ = self.lstm(embeddings)
    outputs = self.linear(hiddens)
    return outputs
  
class ImageCaptionsNet(nn.Module):
  def __init__(self):
    embed_size = 
    hidden_size =  
    vocab_size = 
    num_layers = 
    super(ImageCaptionsNet, self).__init__()
    self.encoderCNN = EncoderCNN(embed_size)
    self.decodeRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

  def forward(self,image_batch, captions_batch):
    features = self.encoderCNN(image_batch)
    outputs = self.decoderRNN(features, captions_batch)
    return outputs

#   def caption_image(self, image, vocabulary, max_length=1000):
#       result_caption = []

#       with torch.no_grad():
#           x = self.encoderCNN(image).unsqueeze(0)
    #       states = None #hidden state for the lstm, in the begining states are none, but leter on it we be change.

    #       for _ in range(max_length):
    #           hiddens, states = self.decoderRNN.lstm(x, states)
    #           output = self.decoderRNN.linear(hiddens.squeeze(0))
    #           predicted =output.argmax(1) 

    #           result_caption.append(predicted.item())
    #           x = self.decoderRNN.embed(predicted).unsqueeze(0)
     
    #           if predicted.item() == 2:
    #               break
    #   return [vocabulary.itos[idx] for idx in result_caption]     


net = ImageCaptionsNet()

# If GPU training is required
# net = net.cuda()




# class ImageCaptionsNet(nn.Module):
#     def __init__(self):
#     # def __init__(self, input_size, hidden_size):
#         super(ImageCaptionsNet, self).__init__()

#         # Define your architecture here

#     def forward(self, x):
#         x = image_batch, captions_batch

#         # Forward Propogation

#         return captions_batch

# net = ImageCaptionsNet()

# # If GPU training is required
# # net = net.cuda()


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

#batch_size = 500   # Number of samples in each batch
#epoch_num = 4      # Number of epochs to train the network
#lr = 0.0005        # Learning rate


class CHHIPA(nn.Module):
    def __init__(self, embed_size):
        super(CHHIPA, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.layer_1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.layer_2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        # max pooling
        self.layer_3 = nn.Conv2d(32, 128, kernel_size=5, padding=2)
        self.layer_4 = nn.Conv2d(128, 128, kernel_size=5, padding=2)
        # max pooling
        self.layer_5 = nn.Conv2d(128, 512, kernel_size=5, padding=2)
        self.layer_6 = nn.Conv2d(512, 512, kernel_size=5, padding=2)
        # max pooling

        self.layer_8 = nn.Conv2d(512, 1024, kernel_size=5, padding=2)
        self.layer_9 = nn.Conv2d(1024, 1024, kernel_size=5, padding=2)
        self.layer_10 = nn.Conv2d(1024, 1024, kernel_size=5, padding=2)
        # max pooling
        self.layer_11 = nn.Conv2d(1024, 1024, kernel_size=5, padding=2)
        self.layer_12 = nn.Conv2d(1024,1024 , kernel_size=5, padding=2)
        # max pooling
        
        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(7*7*1024, 8192)
        self.fc7 = nn.Linear(8192, 8192)
        self.fc8 = nn.Linear(8192, 4096)
        self.fc9 = nn.Linear(4096, embed_size)

    def forward(self, x, training=True):
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = self.pool(x)
        x = F.relu(self.layer_3(x))
        x = F.relu(self.layer_4(x))
        x = self.pool(x)
        x = F.relu(self.layer_5(x))
        x = F.relu(self.layer_6(x))
        x = self.pool(x)
        x = F.relu(self.layer_8(x))
        x = F.relu(self.layer_9(x))
        x = F.relu(self.layer_10(x))
        x = self.pool(x)
        x = F.relu(self.layer_11(x))
        x = F.relu(self.layer_12(x))
        x = self.pool(x)
        x = x.reshape(1,x.reshape(-1).shape[0])
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc8(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc9(x)
        return x

# define the CNN and move the network into GPU
#vgg16 = VGG16(10)
#vgg16.cuda()
#from torchvision import models
model = CHHIPA(1000)
print(model)

CHHIPA(
  (layer_1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_3): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_4): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_5): Conv2d(128, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_6): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_8): Conv2d(512, 1024, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_9): Conv2d(1024, 1024, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_10): Conv2d(1024, 1024, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_11): Conv2d(1024, 1024, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer_12): Conv2d(1024, 1024, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc6): L